In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
import re

In [18]:
if __name__ == '__main__':
    #headers for request
    HEADERS = ({'User-Agent':'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 ', 'Accept-Language': 'en-US, en;q=0.5'})
    #url of the page to be scraped
    URL="https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"

    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data in an amazon page
    soup = BeautifulSoup(webpage.content, "html.parser")
    # Fetch links as List of Tag Objects of all the search results
    comp=soup.find_all("div", attrs={'data-component-type':'s-search-result'})

    #initialize an empty list to store the links
    links_list = []
    #loop for extracting data from Tag Objects 
    for i in comp:
        link = i.find("a", attrs={'class':'a-link-normal s-no-outline'})
        url="https://www.amazon.in" + link.get('href')
        links_list.append(url)

    # Creating a dictionary having the features as keys
    d={"url":[], "description":[], "asin":[], "product description":[], "manufacturer":[], }
    for url in links_list:
        webpage = requests.get(url, headers=HEADERS)
        d["url"].append(url)
        soup=BeautifulSoup(webpage.content,"html.parser")
        try:
            description=soup.find("div", attrs={'id':'feature-bullets'}).text.strip()
        except AttributeError:
            description = ''
        d["description"].append(description)
        try:
            asin = re.search(r"/[dg]p/([^/]+)", url, flags=re.IGNORECASE)
            
        if asin:
            
            d["asin"].append(asin.group(1))
        try:
            productdescription=soup.find("div", attrs={'id':'feature-bullets'}).text.strip()
        except AttributeError:
            productdescription = ''
        d["productdescription"].append(productdescription)     
        try:
            manufacturer=soup.find("div", attrs={'id':'feature-bullets'}).text.strip()
        except AttributeError:
            manufacturer = ''
        d["manufacturer"].append(manufacturer)
        
        sleep(5)
        
    print(d)
    
    # Converting the dictionary into a pandas dataframe
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])

    #saving the dataframe to a csv file
    amazon_df.to_csv("part2.csv", header=True, index=False)
    print("Data saved to csv file")

SyntaxError: expected 'except' or 'finally' block (61650156.py, line 36)